---
title: Getting org units from DHIS2 Web API
short_title: Getting from Web API
---

This page shows practical ways to fetch organisation unit (OU) data from the DHIS2 Web API using `dhis2-python-client` library.

In the **climate-tools** environment, `dhis2-python-client` library is already installed. You only need to come up with your DHIS2 credentials and server address to use this library. Below is a basic usage:

In [1]:
from itertools import islice
import pprint

from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

# Client configuration
cfg = ClientSettings(
  base_url="http://localhost:8080",
  username="admin",
  password="district")

client = DHIS2Client(settings=cfg)
info = client.get_system_info()

# Check if everything is working.
# You should see your current DHIS2 version info.
print("▶ Current DHIS2 version: ", info["version"])

▶ Current DHIS2 version:  2.42.2-SNAPSHOT


Once you are sure to be able to make connection with your DHIS2 instance, you can proceed to fetching orgunits. But first you need to decide which orgunit(s) to fetch. The library offers a couple of handy methods for this. Please see [the readme file](https://github.com/dhis2/dhis2-python-client) for more usage examples of the library.

### List OUs by level (auto‑paged)

In [2]:

print("▶ Fetching orgunits at level 3 (first 10 ous shown as sample):")
for ou in islice(client.get_organisation_units(level=3, fields="id,name", order="name:asc"),10):
    print(f"{ou['id']} · {ou['name']}")
print("… (more not shown)")

▶ Fetching orgunits at level 3 (first 10 ous shown as sample):
YuQRtpLP10I · Badjia
jPidqyo7cpF · Bagruwa
vWbkYPRmKyS · Baoma
dGheVylzol6 · Bargbe
zFDYIgyGmXG · Bargbo
RzKeCma9qb1 · Barri
EB1zRKdYjdY · Bendu Cha
fwH9ipvXde9 · Biriwa
ENHOJz3UH5L · BMC
KKkLOTpMXGV · Bombali Sebora
… (more not shown)


### Fetch all OUs in the system

In [3]:
ous = list(client.get_organisation_units())
print("▶ Total number of ous available: ", len(ous))

▶ Total number of ous available:  1332


`client.get_organisation_units()` returns an `Iterable` and can be converted to a `List` of Dict for further processing. However, please be careful of memory usage if you have a huge number of org units in your system.

While `.get_organisation_units(...)`is a convenient method to faciliate page by page data fetching, the library also allows a raw access where you can pass filters and params direct to DHIS2 API. Below is an example:

### Get a single OU by UID

In [19]:
ou_id = "O6uvpzGd5pu"  # Change this to your desired org unit ID
fields = "id,name,level,parent"
org_unit = client.get("/api/organisationUnits/" + ou_id, params={"fields": "id,name,level,parent"})
print(f"▶ Fetched org unit {ou_id}: {org_unit}")

▶ Fetched org unit O6uvpzGd5pu: {'name': 'Bo', 'parent': {'id': 'ImspTQPwCqd'}, 'id': 'O6uvpzGd5pu', 'level': 2}


### Fetch multiple organisation units by IDs

In [ ]:
# IDs of org units to fetch:
ou_ids = ["ImspTQPwCqd", "O6uvpzGd5pu"]

# Call the raw API with the `ids` as filter
resp = client.get(
    "/api/organisationUnits",
    params={
        "fields": "id,displayName,parent[id,displayName]",
        "filter": "id:in:[" + ",".join(ou_ids) + "]",
    },
)

print("▶ Fetching multiple org units by IDs...")
for ou in resp["organisationUnits"]:
    print(ou["id"], ou["displayName"])

▶ Fetching multiple org units by IDs...
O6uvpzGd5pu Bo
ImspTQPwCqd Sierra Leone
